In [11]:
import random

In [98]:
all_cards = [['Ace', 'Bastoni'], ['Ace', 'Denari'], ['Ace', 'Spade'], ['Ace', 'Coppe'],
             ['2', 'Bastoni'], ['2', 'Denari'], ['2', 'Spade'], ['2', 'Coppe'],
             ['3', 'Bastoni'], ['3', 'Denari'], ['3', 'Spade'], ['3', 'Coppe'],
             ['4', 'Bastoni'], ['4', 'Denari'], ['4', 'Spade'], ['4', 'Coppe'],
             ['5', 'Bastoni'], ['5', 'Denari'], ['5', 'Spade'], ['5', 'Coppe'],
             ['6', 'Bastoni'], ['6', 'Denari'], ['6', 'Spade'], ['6', 'Coppe'],
             ['7', 'Bastoni'], ['7', 'Denari'], ['7', 'Spade'], ['7', 'Coppe'],
             ['Jack', 'Bastoni'], ['Jack', 'Denari'], ['Jack', 'Spade'], ['Jack', 'Coppe'],
             ['Horse', 'Bastoni'], ['Horse', 'Denari'], ['Horse', 'Spade'], ['Horse', 'Coppe'],
             ['King', 'Bastoni'], ['King', 'Denari'], ['King', 'Spade'], ['King', 'Coppe']]

all_cards_set = {tuple(card) for card in all_cards}

def get_card_value(card):
    '''
    card: a list of two elements, the first one is the value, the second one is the suite
    '''
    card_value = card[0]
    if card_value == 'Ace':
        return 11
    elif card_value == '3':
        return 10
    elif card_value == 'King':
        return 4
    elif card_value == 'Horse':
        return 3
    elif card_value == 'Jack':
        return 2
    elif card_value == '7':
        return 0.0007
    elif card_value == '6':
        return 0.0006
    elif card_value == '5':
        return 0.0005
    elif card_value == '4':
        return 0.0004
    elif card_value == '2':
        return 0.0002

def compare_cards(card_first_to_move, card_second_to_move, trump):

    # Get the suites and values of the cards
    trump_suite = trump[1]
    card_first_to_move_suite = card_first_to_move[1]
    card_second_to_move_suite = card_second_to_move[1]
    card_first_to_move_value = get_card_value(card_first_to_move)
    card_second_to_move_value = get_card_value(card_second_to_move)

    # If one card is the trump and the other is not, the trump wins
    if card_first_to_move_suite == trump_suite and card_second_to_move_suite != trump_suite:
        return 1
    elif card_first_to_move_suite != trump_suite and card_second_to_move_suite == trump_suite:
        return 2
    
    # If the suites are the same, the highest value wins
    elif card_first_to_move_suite == card_second_to_move_suite:
        if card_first_to_move_value > card_second_to_move_value:
            return 1
        elif card_first_to_move_value < card_second_to_move_value:
            return 2
        
    # If the suites are different and none of them is the trump, the first one to move wins
    else:
        return 1

In [172]:
class BriscolaGame:

    def __init__(self, Player_1, Player_2, trump):
        self.Player_1 = Player_1
        self.Player_2 = Player_2
        self.trump = trump
        self.deck = all_cards_set.copy()
        self.player_1_points = 0
        self.player_2_points = 0
        self.to_move = 1
        self.round_number = 0

    def get_points(self):
        return self.player_1_points, self.player_2_points
    
    def initial_deck_update(self):
        # remove the cards in the players' hands from the deck
        self.deck = self.deck - self.Player_1.get_hand() - self.Player_2.get_hand() - {self.trump}
    
    def draw_card(self, player):
        # pick a random card from the deck
        try:
            card = random.choice(list(self.deck))
        except:
            card = self.trump
            print('Deck is empty, trump is drawn')

        # add the card to the player's hand
        if player == 1:
            self.Player_1.update_hand(card)
        elif player == 2:
            self.Player_2.update_hand(card)
        else:
            print('Player not recognized')
        
        # remove the card from the deck
        try:
            self.deck.remove(card)
        except:
            pass
    
    def play_round(self):

        # Update the round number
        self.round_number += 1
        # print('Round', self.round_number)

        # The first one to move plays a card
        if self.to_move == 1:
            card_first_to_move = self.Player_1.play_card()
            # print('Player 1 plays', card_first_to_move)
        elif self.to_move == 2:
            card_first_to_move = self.Player_2.play_card()
            # print('Player 2 plays', card_first_to_move)

        # The second one to move plays a card
        if self.to_move == 1:
            card_second_to_move = self.Player_2.play_card(opponent_card = card_first_to_move)
            # print('Player 2 plays', card_second_to_move)
        elif self.to_move == 2:
            card_second_to_move = self.Player_1.play_card(opponent_card = card_first_to_move)
            # print('Player 1 plays', card_second_to_move)

        # Compare the cards
        winner = compare_cards(card_first_to_move, card_second_to_move, self.trump)
        # print('Winner:', winner)

        # Update the points
        if winner == 1:
            self.player_1_points += get_card_value(card_first_to_move) + get_card_value(card_second_to_move)
        elif winner == 2:
            self.player_2_points += get_card_value(card_first_to_move) + get_card_value(card_second_to_move)

        # Update the player to move
        self.to_move = winner

        # Each player draws a card if the deck is not empty
        if self.deck != set():
            self.draw_card(winner)
            self.draw_card(3 - winner)

class Player:
    def __init__(self, hand, trump):
        '''
        hand: set of three cards
        '''
        self.hand = hand
        self.trump = trump
        self.points = 0
        self.opp_points = 0
        self.played_cards = set()

    def get_hand(self):
        return self.hand
    
    def play_card(self, opponent_card = None):
        # to be implemented for each player
        pass
    
    def update_hand(self, card):
        self.hand.add(card)


class RandomPlayer(Player):
    def __init__(self, hand, trump):
        '''
        hand: set of three cards
        '''
        super().__init__(hand, trump)

    def play_card(self, opponent_card = None):
        # Randomly choose a card from the hand
        card = random.choice(list(self.hand))
        self.hand.remove(card)
        return card
    
class GreedyPlayer(Player):
    def __init__(self, hand, trump):
        '''
        hand: set of three cards
        '''
        super().__init__(hand, trump)

    def play_card(self, opponent_card = None):  
        # If the player is the first to move, play the lowest card
        if opponent_card == None:
            max_card = min(self.hand, key = get_card_value)
            self.hand.remove(max_card)
            return max_card
   
        # If the player is the second to move, play the card to maximize the points
        if opponent_card != None:
            # Go through all the cards in the hand
            max_points = -1000
            max_card = None

            for card in list(self.hand):
                # Check who wins if the player plays the card
                if compare_cards(opponent_card, card, self.trump) == 1:
                    # If the opponent wins, minus points
                    current_points = (get_card_value(opponent_card) + get_card_value(card)) * (-1)
                    if current_points > max_points:
                        max_points = current_points
                        max_card = card
                else:
                    # If the player wins, add points
                    current_points = get_card_value(opponent_card) + get_card_value(card)
                    if current_points > max_points:
                        max_points = current_points
                        max_card = card 

            self.hand.remove(max_card)
            return max_card
     


In [192]:
random_hand_1 = set(random.sample(all_cards_set, 3))
print('Player 1 hand:', random_hand_1)
random_hand_2 = set(random.sample(all_cards_set - random_hand_1, 3))
print('Player 2 hand:', random_hand_2)  

Player1 = RandomPlayer(hand=random_hand_1, trump=('Ace', 'Spade'))
Player2 = GreedyPlayer(hand=random_hand_2, trump=('Ace', 'Spade'))
my_game = BriscolaGame(Player1, Player2, trump=('Ace', 'Spade'))

my_game.initial_deck_update()

for i in range(20):
    my_game.play_round()

player_1_final, player_2_final = my_game.get_points()
print('Player 1 points:', player_1_final)
print('Player 2 points:', player_2_final)

if player_1_final > player_2_final:
    print('Player 1 wins')
elif player_1_final < player_2_final:
    print('Player 2 wins')


Player 1 hand: {('7', 'Coppe'), ('Jack', 'Spade'), ('Ace', 'Coppe')}
Player 2 hand: {('Horse', 'Coppe'), ('5', 'Denari'), ('Jack', 'Coppe')}
Deck is empty, trump is drawn
Player 1 points: 68.0061
Player 2 points: 52.0035
Player 1 wins


C:\Users\jakub\AppData\Local\Temp/ipykernel_1464/3997141786.py:1: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_hand_1 = set(random.sample(all_cards_set, 3))
C:\Users\jakub\AppData\Local\Temp/ipykernel_1464/3997141786.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_hand_2 = set(random.sample(all_cards_set - random_hand_1, 3))


In [80]:
compare_cards(('7', 'Coppe'), ('Horse', 'Coppe'), ('Ace', 'Spade'))

TypeError: '>' not supported between instances of 'float' and 'NoneType'